In [1]:
import os
from typing import List, Dict, Optional
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter, NLTKTextSplitter

# Define a base TextSplitter class
class TextSplitter:
    def split_text(self, text: str) -> List[str]:
        raise NotImplementedError("Subclasses should implement this method.")

# Implement specific text splitters
class RecursiveCharacterTextSplitterAdapter(TextSplitter):
    def __init__(self, chunk_size: int = 100, chunk_overlap: int = 20):
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=True
        )

    def split_text(self, text: str) -> List[str]:
        return self.splitter.split_text(text)

class NLTKTextSplitterAdapter(TextSplitter):
    def __init__(self):
        self.splitter = NLTKTextSplitter()

    def split_text(self, text: str) -> List[str]:
        return self.splitter.split_text(text)

# Define a custom embedding function
class CustomSentenceTransformerEmbedding(embedding_functions.EmbeddingFunction):
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        self.model = SentenceTransformer(model_name)

    def __call__(self, texts):
        embeddings = self.model.encode(texts)
        return embeddings.tolist()

# Define the ChromaInterface class
class ChromaInterface:
    def __init__(self, collection_name: str, persist_directory: str, text_splitter: TextSplitter):
        self.client = chromadb.PersistentClient(path=persist_directory)
        self.embedding_function = CustomSentenceTransformerEmbedding()
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.embedding_function
        )
        self.text_splitter = text_splitter

    def add_documents_from_files(self, file_paths: List[str], metadatas: Optional[List[Dict[str, str]]] = None):
        documents = []
        ids = []
        id_counter = 0  # Initialize a counter for unique IDs

        for file_path in file_paths:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Split the content into chunks
                split_texts = self.text_splitter.split_text(content)
                documents.extend(split_texts)
                # Generate unique IDs using a counter
                for _ in split_texts:
                    ids.append(f"{os.path.basename(file_path)}_{id_counter}")
                    id_counter += 1  # Increment the counter for each chunk

        if metadatas is None:
            metadatas = [{"source": file_path} for file_path in file_paths]

        # Adjust metadatas to match the number of document chunks
        extended_metadatas = []
        for i, file_path in enumerate(file_paths):
            metadata = metadatas[i] if metadatas and i < len(metadatas) else {"source": file_path}
            extended_metadatas.extend([metadata] * len(self.text_splitter.split_text(open(file_path, 'r', encoding='utf-8').read())))

        self.collection.add(
            documents=documents,
            metadatas=extended_metadatas,
            ids=ids
        )

    def query(self, query_text: str, n_results: int = 30):
        results = self.collection.query(
            query_texts=[query_text],
            n_results=n_results
        )['documents']
        return results  # List of strings



C:\Users\SAQERpc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Initialize the interface
text_splitter = RecursiveCharacterTextSplitterAdapter(chunk_size=200, chunk_overlap=20)

chroma_interface = ChromaInterface("taw_hist",
                                   "DB/chroma_db",
                                       text_splitter=text_splitter
)





In [2]:
chroma_interface.client.list_collections()


NameError: name 'chroma_interface' is not defined

In [4]:
# Add documents from text files
text_files = ["taw_hist_merged_file.txt"]
chroma_interface.add_documents_from_files(text_files)

In [5]:
len(chroma_interface.collection.get()['ids'])

1235

In [157]:
# Assuming you have already initialized the Chroma collection
chroma_interface.client.delete_collection(name="spain_hist")  # This deletes all documents in the collection



PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'DB/chroma_db\\0c6f6859-c986-4dd1-aa42-d09f9c3138b9\\data_level0.bin'

In [6]:
# Query the collection
#chroma_interface = ChromaInterface("gaza","DB/chroma_db")
query_results = chroma_interface.query("ما هي معركة حطين")
print(query_results) #list of strings


[['الحرب بسرعة الحركة» والانتشار فى يساحات واسعة؛ لإبقاء العدوٌ فى حالة استنفار وضربه فى كلّ مكان؛ لإجهاده؛ وإلحاق أكبر قدر ممكن من الخسائر به» وضرب البنية السّحتيّة له؛ والهجوم بأعداد صغيرة» والقدرة', '* تعريف المقصود بالحروب الفرنجية.\n* توضيح دوافع الحروب الفرنجية.\n* بيان سير أبرز الحملات الفرنجية إلى الشرق.\n* الموازنة بين أخلاقيات القتال عند كل من الفرنجة والمسلمين أثناء الحروب الفرنجية.', 'والثقافية من الهجمات العسكرية. ومن مبادئه:', '* استكشاف نتائج الحروب الفرنجية على الشرق.', 'أساساً على ا( لهجوم المفاجئ» والاختفاء (الكرّ. والفرٌ) . تلجاً إليها القوى الصغيرة» أو الضٌّعيفة العدد والعتاد. عندما لا يكون لديها القدرة على خوض الحرب ضد الجيوش التظاميّة. وأكثر من يلجاً إلى هذا', 'وهناك دوافع أخرى لاندلاع الحروب؛ منها: فشل الحلول السّلمية في حلّ التوتّرات» أو الهروب من الأزمات الدّاخليّة؛ وتصديرها للخارج» أو ازدياد الرّوح العدوانيّة؛ نتيجة لشعور أمّة معيّنة بامتلاكها قدرة', '**المحتويات**\n\nالجزء الأول: فتوحات وحروب عابرة للقارات', '**كيفية تجنب هذه الأماكن ويلات الحروب:**', 'التهيئ

In [128]:
query_results

[['قالب:روابط شقيقة\nقالب:الحرب الفلسطينية الإسرائيلية 2023\nقالب:حروب إسرائيل\nقالب:ضبط استنادي\nقالب:شريط بوابات',
  'وفي 23 يونيو/حزيران، قال رئيس الوزراء الإسرائيلي بنيامين نتنياهو إن «المرحلة الأكثر شدة من القتال ضد حماس في غزة تقترب من نهايتها، مما يسمح لبعض القوات بالانتقال إلى الحدود اللبنانية، حيث تصاعدت عمليات تبادل إطلاق النار مع حزب الله». كما صرح بأنه «منفتح على اتفاق جزئي مع حماس لإعادة بعض الرهائن، لكن الحرب ستستمر بعد فترة توقف من أجل القضاء على حماس»،[391][392] وقالت حماس إن أي اتفاق يجب أن يتضمن وقف إطلاق نار دائم وانسحاب القوات الإسرائيلية من غزة، وأن موقف نتنياهو يؤكد رفضه لاتفاق وقف إطلاق',
  'لاتفاق وقف إطلاق النار الذي طرحته الولايات المتحدة وقرار مجلس الأمن التابع للأمم المتحدة الأخير.[393]',
  'ضمن الاشتباكات الإسرائيلية الفلسطينية 2021 التي عرفت بأحداث الشيخ جراح في القدس، وأخرى كرد على عملية "الفجر الصادق" التي نفذتها إسرائيل على قطاع غزة عام 2022. وتسبب ذلك في دمار للمدنيين من كلا الجانبين، وتزايد عدد القتلى الفلسطينيين. لكن على الرغم من تزايد العنف، وجدت ال